# Backtesting OpenSTEF Models with OpenSTEF-BEAM

In [ ]:
import os

os.environ["OMP_NUM_THREADS"] = "1"  # Set OMP_NUM_THREADS to 1 to avoid issues with parallel execution and xgboost
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"

import logging
import multiprocessing
from datetime import timedelta
from pathlib import Path

from pydantic_extra_types.coordinate import Coordinate
from pydantic_extra_types.country import CountryAlpha2

from openstef_beam.benchmarking.benchmark_pipeline import BenchmarkContext
from openstef_beam.benchmarking.benchmarks.liander2024 import Liander2024Category, create_liander2024_benchmark_runner
from openstef_beam.benchmarking.callbacks.strict_execution_callback import StrictExecutionCallback
from openstef_beam.benchmarking.models.benchmark_target import BenchmarkTarget
from openstef_beam.benchmarking.storage.local_storage import LocalBenchmarkStorage
from openstef_core.types import LeadTime, Q
from openstef_models.integrations.mlflow.mlflow_storage import MLFlowStorage
from openstef_models.presets import (
    ForecastingWorkflowConfig,
    create_forecasting_workflow,
)
from openstef_models.presets.forecasting_workflow import LocationConfig
from openstef_models.workflows import CustomForecastingWorkflow
from openstef_beam.benchmarking.baselines import (
    create_openstef4_preset_backtest_forecaster,
)

logging.basicConfig(level=logging.INFO, format="[%(asctime)s][%(levelname)s] %(message)s")

In [ ]:
OUTPUT_PATH = Path("./benchmark_results")

BENCHMARK_RESULTS_PATH_XGBOOST = OUTPUT_PATH / "XGBoost"
BENCHMARK_RESULTS_PATH_GBLINEAR = OUTPUT_PATH / "GBLinear"
N_PROCESSES = multiprocessing.cpu_count()  # Amount of parallel processes to use for the benchmark

# Model configuration
FORECAST_HORIZONS = [LeadTime.from_string("P3D")]  # Forecast horizon(s)
PREDICTION_QUANTILES = [
    Q(0.05),
    Q(0.1),
    Q(0.3),
    Q(0.5),
    Q(0.7),
    Q(0.9),
    Q(0.95),
]  # Quantiles for probabilistic forecasts

BENCHMARK_FILTER: list[Liander2024Category] | None = None

In [ ]:
common_config = ForecastingWorkflowConfig(
    model_id="common_model_",
    run_name=None,
    model="flatliner",
    horizons=FORECAST_HORIZONS,
    quantiles=PREDICTION_QUANTILES,
    model_reuse_enable=True,
    mlflow_storage=None,
    radiation_column="shortwave_radiation",
    rolling_aggregate_features=["mean", "median", "max", "min"],
    wind_speed_column="wind_speed_80m",
    pressure_column="surface_pressure",
    temperature_column="temperature_2m",
    relative_humidity_column="relative_humidity_2m",
    energy_price_column="EPEX_NL",
    verbosity=0,
)

In [ ]:
xgboost_config = common_config.model_copy(update={"model": "xgboost"})

gblinear_config = common_config.model_copy(update={"model": "gblinear"})

In [ ]:
storage_xgboost = LocalBenchmarkStorage(base_path=BENCHMARK_RESULTS_PATH_XGBOOST)
storage_gblinear = LocalBenchmarkStorage(base_path=BENCHMARK_RESULTS_PATH_GBLINEAR)

In [ ]:

create_liander2024_benchmark_runner(
    storage=storage_xgboost,
    callbacks=[StrictExecutionCallback()],
).run(
    forecaster_factory=create_openstef4_preset_backtest_forecaster(
        workflow_config=xgboost_config,
    ),
    run_name="xgboost",
    n_processes=N_PROCESSES,
    filter_args=BENCHMARK_FILTER,
)

create_liander2024_benchmark_runner(
    storage=storage_gblinear,
    callbacks=[StrictExecutionCallback()],
).run(
    forecaster_factory=create_openstef4_preset_backtest_forecaster(
        workflow_config=gblinear_config,
    ),
    run_name="gblinear",
    n_processes=N_PROCESSES,
    filter_args=BENCHMARK_FILTER,
)

In [ ]:
from openstef_beam.benchmarking import BenchmarkComparisonPipeline
from openstef_beam.benchmarking.benchmarks.liander2024 import LIANDER2024_ANALYSIS_CONFIG

target_provider = create_liander2024_benchmark_runner(
    storage=LocalBenchmarkStorage(base_path=OUTPUT_PATH),
).target_provider

comparison_pipeline = BenchmarkComparisonPipeline(
    analysis_config=LIANDER2024_ANALYSIS_CONFIG,
    storage=LocalBenchmarkStorage(base_path=OUTPUT_PATH),
    target_provider=target_provider,
)
comparison_pipeline.run(run_data={
    "xgboost": storage_xgboost,
    "gblinear": storage_gblinear,
})


In [ ]:
import webbrowser
import os

filepath = os.path.abspath('./benchmark_results/XGBoost/analysis/global/D-1T06:00/rCRPS_grouped.html')
webbrowser.open(f'file://{filepath}')
